## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-10-21-51-00 +0000,nyt,"Gaza Cease-Fire Begins, Israel Says, as Thousa...",https://www.nytimes.com/2025/10/10/world/middl...
1,2025-10-10-21-48-57 +0000,nyt,"Overlooked No More: Violeta Parra, Folk ‘Geniu...",https://www.nytimes.com/2025/10/10/obituaries/...
2,2025-10-10-21-47-10 +0000,nyt,"Holding Back Tears, Prince William Shows a New...",https://www.nytimes.com/2025/10/10/world/europ...
3,2025-10-10-21-47-00 +0000,wsj,"Opinion | Trump, Merit and MIT",https://www.wsj.com/opinion/trump-merit-and-mi...
4,2025-10-10-21-46-58 +0000,nyt,María Corina Machado Wins Nobel Peace Prize De...,https://www.nytimes.com/2025/10/10/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,56
0,gaza,28
35,peace,25
34,nobel,24
36,prize,22


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
186,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...,196
4,2025-10-10-21-46-58 +0000,nyt,María Corina Machado Wins Nobel Peace Prize De...,https://www.nytimes.com/2025/10/10/world/ameri...,168
120,2025-10-10-17-17-55 +0000,nypost,Nobel Peace Prize winner Maria Corina Machado ...,https://nypost.com/2025/10/10/us-news/nobel-pe...,157
91,2025-10-10-18-33-53 +0000,bbc,As Nobel Prize goes to Venezuela's opposition ...,https://www.bbc.com/news/articles/crmegp9lkjro...,144
35,2025-10-10-21-03-30 +0000,bbc,White House blasts Nobel Committee for not awa...,https://www.bbc.com/news/articles/c7842qg15p6o...,142


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
186,196,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...
350,92,2025-10-09-22-34-27 +0000,nypost,American military to oversee initial Gaza peac...,https://nypost.com/2025/10/09/us-news/american...
224,78,2025-10-10-11-33-02 +0000,nyt,"How Letitia James, Now Indicted, Will Stay as ...",https://www.nytimes.com/2025/10/10/us/politics...
26,66,2025-10-10-21-13-00 +0000,wsj,"House Speaker Mike Johnson (R., La.) received ...",https://www.wsj.com/politics/policy/mike-johns...
60,51,2025-10-10-20-09-27 +0000,nypost,US signs massive $6.1B deal with Finland to ca...,https://nypost.com/2025/10/10/world-news/us-si...
330,48,2025-10-09-23-54-00 +0000,wsj,The Bureau of Labor Statistics will bring some...,https://www.wsj.com/politics/policy/federal-go...
182,46,2025-10-10-14-45-15 +0000,latimes,"Sister Jean, nun who became a national icon du...",https://www.latimes.com/sports/story/2025-10-1...
55,35,2025-10-10-20-16-02 +0000,nyt,Explosion at Tennessee Ammunition Plant Leaves...,https://www.nytimes.com/live/2025/10/10/us/exp...
340,33,2025-10-09-23-19-24 +0000,cbc,Judge temporarily blocks National Guard troop ...,https://www.cbc.ca/news/world/national-guard-d...
0,33,2025-10-10-21-51-00 +0000,nyt,"Gaza Cease-Fire Begins, Israel Says, as Thousa...",https://www.nytimes.com/2025/10/10/world/middl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
